##**Detect the Top 10 fashion objects**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/ColabNotebooks/Dataset/RMgx_data/Data.zip" -d "/content/" 

##**Creating a directory of randomly picked 100 images from test data set**

In [3]:
!mkdir detect
!mkdir detect/test

In [4]:
import glob
import shutil

n = 0
files=[]
list_files = glob.glob('/content/Data/test/*')
for i in list_files:
  if n<1000:
    shutil.copy(i, '/content/detect/test')
  else:break
  n+=1

##**Loading the pre-trained VGG-19 model**

In [5]:
from keras.applications.vgg19 import VGG19
model = VGG19()
model.summary()

574726144/574710816 [==============================] - 4s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [ ]:
#converting the image filename into absolute paths
import os
images = os.listdir("/content/detect/test") 
file_names = []
for i in images:
  file_names+= ['/content/detect/test/'+i]
print(file_names)

##**Making predictions from the model**

In [7]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
import pandas as pd

df = pd.DataFrame(columns = ['imageId', 'predictions'])
obj_dict = {}
for img in file_names:
  img_name = img.split("/")[-1]

  # Load the image
  image = load_img(img, target_size=(224, 224))

  # convert image to numpy array
  image = img_to_array(image)

  # reshape the model from 3d to 4d by adding an extra dimension
  # because vgg19 model requires 4-dimensional inputs
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

  # Preprocessing the image data
  image = preprocess_input(image)

  # making predictions
  pred = model.predict(image)

  #decode into normal format
  label = decode_predictions(pred)
  label = label[0][0]
  df = df.append({'imageId': img_name, 'predictions': [label[1], '(%.2f%%)'%(label[2]*100)]}, ignore_index = True)

49152/35363 [=========================================] - 0s 0us/step


In [8]:
df

,imageId,predictions
0,4743.jpg,"[military_uniform, (41.39%)]"
1,3670.jpg,"[stole, (67.28%)]"
2,8499.jpg,"[poncho, (46.32%)]"
3,7932.jpg,"[sarong, (98.78%)]"
4,7637.jpg,"[swimming_trunks, (51.34%)]"
...,...,...
995,6959.jpg,"[maillot, (53.15%)]"
996,4849.jpg,"[gown, (74.66%)]"
997,6510.jpg,"[maillot, (18.88%)]"
998,9466.jpg,"[velvet, (19.11%)]"


In [9]:
#saving the predictions to csv file
df.to_csv('/content/drive/MyDrive/detections.csv', index = False)